# Exploratory Analysis of NBA API available data
## Goal: Create shot charts and investigate other available data to perform spatial analysis

In [20]:
import pandas as pd 
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats

### Function to get player specific shot chart dataframe

In [21]:
def get_player_shotchartdetail(player_name, season_id):
    # player dictionary
    nba_players = players.get_players()
    player_dict = [player for player in nba_players if player['full_name'] == player_name][0]
    
    # career df
    career = playercareerstats.PlayerCareerStats(player_id=player_dict['id'])
    career_df = career.get_data_frames()[0]
    
    # team id during the season
    team_id = career_df[career_df['SEASON_ID'] == season_id]['TEAM_ID']
    
    # shotchardtdetail endpoint
    shotchartlist = shotchartdetail.ShotChartDetail(team_id=int(team_id), 
                                                   player_id=int(player_dict['id']), 
                                                   season_type_all_star='Regular Season', 
                                                   season_nullable=season_id,
                                                   context_measure_simple="FGA").get_data_frames()
    
    return shotchartlist[0], shotchartlist[1]

Get shot chart info for Kawhi Leanords 2018-2019 season

In [22]:
player_shotchart_df, league_avg = get_player_shotchartdetail('Kawhi Leonard', '2018-19')
player_shotchart_df.head()

,GRID_TYPE,GAME_ID,GAME_EVENT_ID,PLAYER_ID,PLAYER_NAME,TEAM_ID,TEAM_NAME,PERIOD,MINUTES_REMAINING,SECONDS_REMAINING,...,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,SHOT_DISTANCE,LOC_X,LOC_Y,SHOT_ATTEMPTED_FLAG,SHOT_MADE_FLAG,GAME_DATE,HTM,VTM
0,Shot Chart Detail,0021800008,9,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,11,24,...,Center(C),16-24 ft.,21,13,214,1,0,20181017,TOR,CLE
1,Shot Chart Detail,0021800008,28,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,9,47,...,Right Side(R),8-16 ft.,12,118,44,1,0,20181017,TOR,CLE
2,Shot Chart Detail,0021800008,45,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,8,31,...,Center(C),8-16 ft.,12,-34,120,1,0,20181017,TOR,CLE
3,Shot Chart Detail,0021800008,53,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,8,3,...,Center(C),Less Than 8 ft.,1,2,12,1,1,20181017,TOR,CLE
4,Shot Chart Detail,0021800008,98,202695,Kawhi Leonard,1610612761,Toronto Raptors,1,4,44,...,Center(C),Less Than 8 ft.,4,0,41,1,1,20181017,TOR,CLE


Function also returned league averages for the given season based on zones

In [23]:
league_avg

,GRID_TYPE,SHOT_ZONE_BASIC,SHOT_ZONE_AREA,SHOT_ZONE_RANGE,FGA,FGM,FG_PCT
0,League Averages,Above the Break 3,Back Court(BC),Back Court Shot,43,2,0.047
1,League Averages,Above the Break 3,Center(C),24+ ft.,15944,5449,0.342
2,League Averages,Above the Break 3,Left Side Center(LC),24+ ft.,22539,7944,0.352
3,League Averages,Above the Break 3,Right Side Center(RC),24+ ft.,21759,7666,0.352
4,League Averages,Backcourt,Back Court(BC),Back Court Shot,466,14,0.030
5,League Averages,In The Paint (Non-RA),Center(C),8-16 ft.,11502,4866,0.423
6,League Averages,In The Paint (Non-RA),Center(C),Less Than 8 ft.,19554,7665,0.392
7,League Averages,In The Paint (Non-RA),Left Side(L),8-16 ft.,1999,797,0.399
8,League Averages,In The Paint (Non-RA),Right Side(R),8-16 ft.,2067,849,0.411
9,League Averages,Left Corner 3,Left Side(L),24+ ft.,9112,3519,0.386


Check for missing values in XY coordinates

In [30]:
player_shotchart_df['LOC_X'].isnull().any() or player_shotchart_df['LOC_Y'].isnull().any()

False

### Shot Chart Creation